## 3. Normalised Price Stability by Suburb

from previous:
* Geocode addresses to identify nearby petrol stations, and create master list of station -> station and distance.
* Find combinations of nearby stations with a consistent price discrepancy on the same fuel
* Generate infographic of prominent station pairs/fuels

6. avg st dev's within each suburb / fuel
7. calculate range between avg normal price for each station within each suburb / fuel
8. view largest ranges results

# Step 1: Preparing the data


To get started, let's load up our daily pricing data and station master data:

In [1]:
import pandas as pd

daily_pricing = pd.read_csv('../data/daily_pricing.csv.gz', compression='gzip')
station_master = pd.read_csv('../data/station_master.csv.gz', compression='gzip')

From the station master data, we will create a map from `ServiceStationName` -> `Suburb`, then join it onto the daily pricing data:

In [4]:
suburb_map = (
    station_master[['ServiceStationName','Suburb']]
    .groupby(['ServiceStationName','Suburb'])
    .count()
    .reset_index()
)

daily_pricing_suburb = pd.merge(left=daily_pricing, right=suburb_map, on='ServiceStationName', how='left')
daily_pricing_suburb.sample(10, random_state=1234)

,FuelCode,ServiceStationName,Date,Price,Suburb
3614841,U91,BP Nyngan,2016-12-10,129.7,Nyngan
1331959,LPG,BP Smeaton Grange,2016-08-03,54.9,Smeaton Grange
1722259,P95,BP Kemps Creek,2016-12-28,136.9,Kemps Creek
3471628,PDL,Woolworths Park Beach Plaza,2017-10-17,124.9,Coffs Harbour
2112031,P95,Independent Sandgate,2017-01-28,131.7,Sandgate
3704038,U91,Caltex Ermington,2017-07-12,129.9,ERMINGTON
2486773,P98,Budget Petrol Matraville,2016-11-07,141.9,Matraville
1065670,E10,Mobil Crookwell,2017-10-26,129.9,Crookwell
2955604,P98,United Turvey Park,2017-07-17,145.0,Turvey Park
2975468,P98,Woolworths Campbelltown,2017-10-19,154.4,Campbelltown


Now for each suburb and fuel code, we'll calculate the minimum daily price. We will use this to normalise prices within suburbs, and so we map back to `daily_pricing_suburb`:

In [5]:
daily_min_pricing_suburb = (
    daily_pricing_suburb[['Suburb','FuelCode','Date','Price']]
    .groupby(['Suburb','FuelCode','Date'])
    .min()
    .reset_index()
)

daily_norm_pricing = pd.merge(left=daily_pricing_suburb,
         right=daily_min_pricing_suburb,
         on=['Suburb','FuelCode','Date'],
         how='left',
         suffixes=('','_min'))
daily_norm_pricing['Price_norm'] = daily_norm_pricing['Price'] - daily_norm_pricing['Price_min']

daily_norm_pricing.sample(10, random_state=1234)

,FuelCode,ServiceStationName,Date,Price,Suburb,Price_min,Price_norm
3614841,U91,BP Nyngan,2016-12-10,129.7,Nyngan,129.7,0.0
1331959,LPG,BP Smeaton Grange,2016-08-03,54.9,Smeaton Grange,54.9,0.0
1722259,P95,BP Kemps Creek,2016-12-28,136.9,Kemps Creek,135.2,1.7
3471628,PDL,Woolworths Park Beach Plaza,2017-10-17,124.9,Coffs Harbour,124.9,0.0
2112031,P95,Independent Sandgate,2017-01-28,131.7,Sandgate,131.7,0.0
3704038,U91,Caltex Ermington,2017-07-12,129.9,ERMINGTON,129.9,0.0
2486773,P98,Budget Petrol Matraville,2016-11-07,141.9,Matraville,141.9,0.0
1065670,E10,Mobil Crookwell,2017-10-26,129.9,Crookwell,129.9,0.0
2955604,P98,United Turvey Park,2017-07-17,145.0,Turvey Park,145.0,0.0
2975468,P98,Woolworths Campbelltown,2017-10-19,154.4,Campbelltown,137.4,17.0


With a normalised daily price (at the suburb level), for each service station and fuel code we're going to calculate the standard deviation of this normalised price.

A standard deviation of zero means that the price is always the same distance from the minimum price in that suburb. As the standard deviation increases, so too does the fluctation in distance.

Note that the standard deviation doesn't tell us which stations are more or less expensive, just that their prices are stable (or not).

In [12]:
station_price_instability = (
    daily_norm_pricing[ ['Suburb','ServiceStationName','FuelCode','Price_norm']]
    .groupby(['Suburb','ServiceStationName','FuelCode'])
    .std()
    .reset_index()
    .rename(columns={'Price_norm': 'Price_instability'})
)

station_price_instability.sample(10, random_state=1234)

,Suburb,ServiceStationName,FuelCode,Price_instability
8339,SILVERWATER,BP Silverwater,DL,0.000000
9314,Thornleigh,Caltex Thornleigh,P95,4.223412
1435,Budgewoi,United Petroleum Budgewoi,E85,0.000000
7389,PORT MACQUARIE,Caltex Port Macquarie,E10,0.000000
6963,Neutral Bay,Coles Express Neutral Bay,P98,2.704095
545,Bankstown,7-Eleven Bankstown,P98,7.372491
8284,Rydalmere,Caltex Rydalmere,LPG,1.089500
1978,Casula,BP Casula,E10,9.493309
8856,Stanford Merthyr,BP Stanford Merthyr,U91,0.000000
1544,CAMBRIDGE PARK,Caltex Cambridge,E10,0.000000


Now, for each service station and fuel code we're going to calculate the average of this normalised price to discover which stations are more or less expensive.

In [13]:
station_price_average = (
    daily_norm_pricing[['Suburb','ServiceStationName','FuelCode','Price_norm']]
    .groupby(['Suburb','ServiceStationName','FuelCode'])
    .mean()
    .reset_index()
    .rename(columns={'Price_norm': 'Price_average'})
)

station_price_average.sample(10, random_state=1234)

,Suburb,ServiceStationName,FuelCode,Price_average
8339,SILVERWATER,BP Silverwater,DL,0.000000
9314,Thornleigh,Caltex Thornleigh,P95,1.615098
1435,Budgewoi,United Petroleum Budgewoi,E85,0.000000
7389,PORT MACQUARIE,Caltex Port Macquarie,E10,0.000000
6963,Neutral Bay,Coles Express Neutral Bay,P98,3.675274
545,Bankstown,7-Eleven Bankstown,P98,4.858206
8284,Rydalmere,Caltex Rydalmere,LPG,0.459823
1978,Casula,BP Casula,E10,13.326570
8856,Stanford Merthyr,BP Stanford Merthyr,U91,0.000000
1544,CAMBRIDGE PARK,Caltex Cambridge,E10,0.000000


Finally, we're going to find the max and min normal average prices per suburb, calculate the effective range, and take the mean price instability to look for lower values (representing greater consistency over the analysis period).

Then we are ready to start interpreting our results.

In [48]:
station_pricing = pd.merge(left=station_price_instability,
         right=station_price_average,
         on=['Suburb','ServiceStationName','FuelCode'],
         how='inner')

suburb_range = (
    station_pricing[['Suburb','FuelCode','Price_average','Price_instability']]
    .groupby(['Suburb','FuelCode'])
    .agg({'Price_average': ['max','min'], 'Price_instability': ['mean']})
    .reset_index()
)

# flatten the multi-index created by .agg() above
suburb_range.columns = ['_'.join(col).strip('_') for col in suburb_range.columns.values]

suburb_range['Price_range'] = suburb_range['Price_average_max'] - suburb_range['Price_average_min']

suburb_range.sample(10, random_state=1234)

,Suburb,FuelCode,Price_instability_mean,Price_average_max,Price_average_min,Price_range
1339,Coopernook,P98,0.000000,0.000000,0.000000,0.000000
154,Arndell Park,LPG,0.000000,0.000000,0.000000,0.000000
884,CARLINGFORD,P95,0.386842,0.096491,0.018519,0.077973
3863,North Ballina,PDL,0.000000,0.000000,0.000000,0.000000
4649,Seven Hills,U91,6.174448,12.810746,1.559341,11.251405
4921,TABULAM,DL,0.000000,0.000000,0.000000,0.000000
2438,Hillvue,P98,0.000000,0.000000,0.000000,0.000000
516,Bendalong,P95,0.000000,0.000000,0.000000,0.000000
3102,Liverpool,U91,6.185492,18.397593,0.900223,17.497370
240,BELLAMBI,E10,0.000000,0.000000,0.000000,0.000000


# Step 2: Interpreting the data

Let's now examine our results!

Firstly, we're going to filter to values of `FuelCode` that represent unleaded fuels. (Diesel fuels and other alternatives can be the subject of a separate analysis.)

We're also going to filter where `Price_range` is 50 or greater (this is most likely an error due to static pricing forward fill done in the previous notebook).

In [79]:
(
    suburb_range[
        suburb_range['FuelCode'].isin(['E10','U91','P95','P98']) & 
        (suburb_range['Price_range'] < 50) &
        (suburb_range['Price_range'] > 0)
    ]
    .sort_values(by='Price_range', ascending=False)
)

,Suburb,FuelCode,Price_instability_mean,Price_average_max,Price_average_min,Price_range
5043,Tamworth,P95,6.094812,39.717084,0.924123,38.792961
2354,Hamilton,P98,0.696020,32.444444,0.000000,32.444444
2350,Hamilton,E10,0.696020,32.444444,0.000000,32.444444
2353,Hamilton,P95,0.696020,32.444444,0.000000,32.444444
5934,Woy Woy,P98,7.899863,32.057743,0.000000,32.057743
1016,Camden,U91,6.308407,26.647692,0.000000,26.647692
2343,Haberfield,P98,8.062587,26.460394,0.802368,25.658025
1010,Camden,E10,10.129176,25.064176,0.296499,24.767677
3516,Miranda,U91,7.633122,25.231072,0.533479,24.697593
3511,Miranda,E10,7.643044,24.804814,0.590810,24.214004
